In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np
import re
from selenium.common.exceptions import TimeoutException
import time

In [3]:
def get_categories(link):
    result = None

    while result is None:
        try:
            driver = webdriver.Chrome(executable_path='chromedriver.exe')
            driver.get(link)
            content = driver.page_source
            soup = BeautifulSoup(content)
            gui=soup.find_all(class_="swiper-wrapper tabs-items")
            if len(gui)!=0:
                li=re.findall(',([\w+ ]*)', gui[0].text.replace('\n', ',') )[:-1]
                ids = [tag['onclick'] for tag in gui[0].select('div[id]')]
                tags = [j.split("(",1)[1].split(",",1)[0].split("'")[1] for j in ids]
            
            else:
                li=[]
                tags=[]
            result=1
            driver.quit()
        except:
            time.sleep(10)
            driver.quit()
            pass
    return li, tags, soup


In [4]:
def get_births(link, tex, tag, lol):
    common_data = [lol,link]
    df=pd.DataFrame(columns=['name', 'position', 'dob1', 'matches', 'minutes', 'goals', 'dob2', 'link', 'category'])
    soup = tex
    #match = soup.find('div', {'id': tag})
    string='plantel_'+str(tag[4:])
    match2 = soup.find('div', {'id': string})
    gui=match2.find_all(class_="row justify-content-between align-items-center")
    for j in range(0, len(gui)):
        data=gui[j].text.replace('\n', ',').replace('\t', ',').split(',')
        if len(list(filter(None, data)))==7:
            df.loc[len(df)] = list(filter(None, data)) + common_data
        else:
            df.loc[len(df)] =  [list(filter(None, data))[0]] + [''] + list(filter(None, data))[1:] + common_data
    result = 1
    return df

In [5]:
dictio={"Nacional": 81,
"River Plate": 4463,
"D. Maldonado":240,
"Danubio": 284,
"Liverpool": 205,
"Def. Sporting": 190,
"Peñarol": 139,
"Cerro Largo": 92,
"Boston River": 20,
"Fénix": 25,
"Albion": 150,
"Wanderers": 231,
"Plaza Colonia": 137,
"M.C. Torque": 105,
"Rentistas": 156,
"Cerrito": 59,
"Racing": 4471,
"Uruguay Montevideo": 280,
"Cerro": 226,
"La Luz": 79,
"Miramar Misiones": 91,
"Sud América": 127,
"Atenas de San Carlos": 187,
"Rampla Juniors": 4462,
"Villa Española": 65,
"Progreso": 89,
"Juventud": 202,
"Central Español": 265,
"Nacional (F)": 273,
"Peñarol (F)": 68,
"Defensor Sporting (F)": 236,
"Wanderers (F)": 243,
"Liverpool (F)": 61,
"Fénix (F)": 72,
"River Plate (F)": 229,
"Náutico (F)": 304,
"Atenas (F)": 73,
"M.C. Torque (F)": 4694,
"Danubio  (F)": 174,
"S.J. Rentistas (F)": 99,
"Boston River (F)": 256,
"Racing (F)": 83,
"Canadian (F)": 222,
"Juventud (F)": 122,
"Rampla Juniors (F)": 257,
"Plaza Colonia (F)": 307
}


In [9]:
df2=pd.DataFrame()
for j in list(dictio):
    link1='https://www.auf.org.uy/historico-planteles/'+str(dictio[j])+'/0/'
    di=get_links(link1)
    linklist=['/historico-planteles/'+str(dictio[j])+'/0/']
    for z in range(0,len(di)):
        for a in di[z]('a', href=True):
            linklist=linklist+[a['href']]
    for n in linklist:
        link1='https://www.auf.org.uy'+n
        lez,per,tex = get_categories(link1)
        if (len(lez)!=0) & (len(per)!=0):
            for t in per:
                w=per.index(t)
                rom=get_births(link1, tex,t,lez[w])
                df2=df2.append(rom)
    print('Last Team is ', j, '. The number of players is: ', len(df2))

C:\Users\guill\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


Last Team is  Wanderers . The number of players is:  2548
Last Team is  Plaza Colonia . The number of players is:  4955
Last Team is  M.C. Torque . The number of players is:  6779
Last Team is  Rentistas . The number of players is:  9309
Last Team is  Cerrito . The number of players is:  11347
Last Team is  Racing . The number of players is:  13805
Last Team is  Uruguay Montevideo . The number of players is:  14319
Last Team is  Cerro . The number of players is:  16756
Last Team is  La Luz . The number of players is:  17925
Last Team is  Miramar Misiones . The number of players is:  20095
Last Team is  Sud América . The number of players is:  22294
Last Team is  Atenas de San Carlos . The number of players is:  24375
Last Team is  Rampla Juniors . The number of players is:  26602
Last Team is  Villa Española . The number of players is:  28370
Last Team is  Progreso . The number of players is:  30348
Last Team is  Juventud . The number of players is:  32844
Last Team is  Central Español

In [ ]:
df2.to_csv('data/final_auf.csv)